# Задание 3
1. Изучить на скрытые темы “Преступление и наказание” Реализовать тематическое моделирование через кластеризацию и BERTopic
2. Сравнить эффективность  использования евклидова расстояния и 2 других типов расстояния в кластеризации
3. Визуализировать кластера по выделенным топикам
4. Выложить на github

## Предобработка текста

In [1]:
!gdown 1swDS6kMKlU1EJ5VlxkDeNENnHjNSw5Zi
!pip install bertopic
!pip install umap-learn
!pip install hdbscan
!pip install bertopic
!pip install sentence-transformers
!pip install pymystem3
!pip install -U plotly==5.19.0
import nltk
nltk.download('stopwords')

Downloading...
From: https://drive.google.com/uc?id=1swDS6kMKlU1EJ5VlxkDeNENnHjNSw5Zi
To: /content/Crime And Punishment.txt
100% 1.93M/1.93M [00:00<00:00, 88.8MB/s]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
import pandas as pd
from nltk import tokenize
import nltk
nltk.download('punkt')

articles = []
chapters = []
with open('Crime And Punishment.txt', "r") as file:
    chapters = "".join("".join(file.readlines()).split("* * *")[2:])
    chapters = re.split(r'(\r\n|\r|\n)+[MDCLXVI]+(\r\n|\r|\n)', string=chapters, flags=re.IGNORECASE)
    chapters = [chapter for chapter in chapters if len(chapter) > 25]
    articles = [article for chapter in chapters for article in chapter.split("\n") if len(article) > 20]
    sentences = [sentence for article in articles for sentence in tokenize.sent_tokenize(article) if len(sentence) > 5]

    chapters, articles, sentences = pd.DataFrame({"text": chapters}), pd.DataFrame({"text": articles}), pd.DataFrame({"text": sentences})

print(len(chapters))
print(len(articles))
print(len(sentences))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


41
3528
13403


In [3]:
from pymystem3 import Mystem
import string
import nltk
from nltk.corpus import stopwords
def standartize_text(df):
    replacements = {
      "счастие": "счастье",
      "счастия": "счастье",
      "счастии": "счастье",
      "жадностию": "жадностью",
      "частию": "частью",
      "торжественностию": "торжественностью",
      "покиванием": "киванием",
      "глупостию": "глупостью",
      "хошь": "хочешь",
      "полицу": "полицию",
      "скрыпя": "скрипя",
      "пошто": "почему",
      "каку": "какую",
      "милостию": "милостью",
      "матерчатое": "пустоту",
      "выпуклостию": "выпуклостью",
      "точностию": "точностью",
      "помошию": "помощью",
      "мыслию": "мыслью",
      "кровию": "кровью",
      "непочтительностию": "небрежностию",
      "што": "что",
      "жизнию": "жизнью",
      "чё": "что",
      "нє": "не",
      "тутто": "тут",
      "зде": "здесь",
      "ведаю": "знаю",
      "пошёл": "пошел",
      "приходшу": "прихожу",
      "делаю": "делаю",
      "быстр": "быстрый",
      "нов": "новый",
      "стар": "старый",
      "мал": "маленький",
      "велик": "великий",
      "вещица": "вещь",
      "молвить": "сказать",
      "глаголить": "говорить",
      "сия": "эта",
      "оную": "эту",
      "несть": "нет",
      "ежели": "если",
      "каков": "какой",
      "оне": "они",
      "токмо": "только",
      "аз": "я",
      "буй": "бой",
      "веде": "знает",
      "вѣрно": "верно",
      "мѣсто": "место",
      "рѣка": "река",
      "лѣто": "лето",
      "дѣло": "дело"
      }
    pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in replacements.keys()) + r')\b')
    df.apply(lambda text: pattern.sub(lambda x: replacements[x.group()], text))
    m = Mystem()
    sw = stopwords.words("russian")

    df = df.replace(to_replace=r'[^\w\s]', value=" ", regex=True)
    df = df.replace(to_replace=r"(.)\1\1+", value=r'\1\1', regex=True)
    list_patterns = [r'\b[^\W\dА-Яа-яёЁ]+\b', r'[string.punctuation]', '/\r?\n|\r/', '\n']
    pattern = re.compile("|".join(list_patterns))
    df = df.replace(to_replace=pattern, value='', regex=True)
    df = df.replace(to_replace=r'\s+', value=' ', regex=True).str.strip()
    df = df.apply(lambda x: " ".join([word for word in m.lemmatize(x) if word not in sw]))
    df = df.str.strip()
    return df

chapters['text'] = standartize_text(chapters['text'])
articles['text'] = standartize_text(articles['text'])
sentences['text'] = standartize_text(sentences['text'])

## Выберем на данных какого вида будем обучать BERTtopic

In [4]:
from bertopic import BERTopic
from nltk.corpus import stopwords
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from transformers.pipelines import pipeline
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

from sentence_transformers import SentenceTransformer

import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
embedding_model = SentenceTransformer('ai-forever/sbert_large_mt_nlu_ru')
umap_model = UMAP(n_components=5, metric='cosine', n_neighbors=15, min_dist=0.0)
hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True, prediction_data=True)
vectorizer_model = CountVectorizer(stop_words=stopwords.words('russian') + stopwords.words('english'))
ctfidf_model = ClassTfidfTransformer()
representation_model = KeyBERTInspired()
def create_topic_model(docs):
    return BERTopic(language="russian", embedding_model=embedding_model, umap_model=umap_model, hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model, ctfidf_model=ctfidf_model, representation_model=representation_model).fit(docs)


### Деление на главы

Как видим деление на главы не имеет смысла, т.к. модель не может выделить темы для такого маленького обьема данных. Да и у нас есть ограничесние на 512 токенов, так что большая часть главы обрезается

In [5]:
chapter_model = create_topic_model(chapters['text'].to_list())
chapter_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,41,-1_тотчас_случай_понимать_проговаривать,"[тотчас, случай, понимать, проговаривать, дава...",[раскольников пойти вслед это вс...


### Деление на абзацы

In [6]:
article_model = create_topic_model(articles['text'].to_list())
article_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2226,-1_случай_тотчас_проговаривать_понимать,"[случай, тотчас, проговаривать, понимать, мысл...",[раскольник уставать весь время ...
1,0,61,0_отворять_входить_проходить_отпирать,"[отворять, входить, проходить, отпирать, проск...",[полный отчаяние пойти прямо навст...
2,1,60,1_спрашивать_задавать_вопрос_отвечать,"[спрашивать, задавать, вопрос, отвечать, выпыт...","[отвечать спрашивать дуня, сказать спраш..."
3,2,49,2_соня_сонечка_прошептать_пойти,"[соня, сонечка, прошептать, пойти, вскричать, ...","[н прошептать соня, пойти соня, ..."
4,3,47,3_лгать_солгать_врать_клеветник,"[лгать, солгать, врать, клеветник, признаватьс...",[лгать бешенство засверкать глаз дун...
5,4,47,4_александровна_пульхерия_порфирий_вскричать,"[александровна, пульхерия, порфирий, вскричать...",[боже вскричать пульхерия александровн...
6,5,42,5_раскольников_раскольник_прерываться_замалчивать,"[раскольников, раскольник, прерываться, замалч...",[дело приходить говорить громко нахм...
7,6,40,6_отворять_отпирать_запираться_запирать,"[отворять, отпирать, запираться, запирать, две...","[дверь, ключ отворять дверь низкий ..."
8,7,36,7_входить_отпирать_комната_запирать,"[входить, отпирать, комната, запирать, каморка...",[оказываться минута порфирий петро...
9,8,35,8_смысл_ничто_грех_предрассудочный,"[смысл, ничто, грех, предрассудочный, взбесить...",[стоять принимать соображение ош...


### Деление на предложения

In [7]:
sentence_model = create_topic_model(sentences['text'].to_list())
sentence_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,7034,-1_входить_господин_свидригайлов_ивановна,"[входить, господин, свидригайлов, ивановна, ко...","[весь это, ваш сестрица авдотья романо..."
1,0,335,0____,"[, , , , , , , , , ]","[, , ]"
2,1,155,1_отворять_запираться_отворяться_отпирать,"[отворять, запираться, отворяться, отпирать, з...","[замечать дверь запирать ключ, сюд..."
3,2,105,2_крикнуть_вскричать_закричать_прошептать,"[крикнуть, вскричать, закричать, прошептать, к...","[крикнуть разумихин, крикнуть разумихин, к..."
4,3,97,3_выдерживать_удерживаться_сдерживать_затвержи...,"[выдерживать, удерживаться, сдерживать, затвер...","[выдерживать выдерживать, выдерживать,..."
...,...,...,...,...,...
250,249,5,249_прочитывать_подписываться_письмо_написать,"[прочитывать, подписываться, письмо, написать,...",[подписываться едва перо водить заме...
251,250,5,250_убирать_убираться_беспорядок_заменять,"[убирать, убираться, беспорядок, заменять, при...","[убирать, настасья прибивать в..."
252,251,5,251_взгляд_голубой_глаз_лицо,"[взгляд, голубой, глаз, лицо, зеркало, черный,...",[глаз слишком голубой взгляд ...
253,252,5,252_надевать_одеваться_пальто_пришивать,"[надевать, одеваться, пальто, пришивать, шляпк...",[пальто спрятать весь таки рука ...


## Рассмотрим несколько видов расстояний кластеризации

In [11]:
from bertopic import BERTopic
from nltk.corpus import stopwords
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from transformers.pipelines import pipeline
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

from sentence_transformers import SentenceTransformer

import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
embedding_model_sbert = SentenceTransformer('ai-forever/sbert_large_mt_nlu_ru')
umap_model_cosine = UMAP(n_neighbors=30, n_components=4, min_dist=0.0, metric='cosine')
HDBSCAN_euclidean = HDBSCAN(min_cluster_size=15, metric='euclidean' , cluster_selection_method='eom', prediction_data=True)
HDBSCAN_chebyshev = HDBSCAN(min_cluster_size=15 , cluster_selection_method='eom', prediction_data=True, metric='chebyshev')
HDBSCAN_manhattan = HDBSCAN(min_cluster_size=15 , cluster_selection_method='eom', prediction_data=True, metric='manhattan')
HDBSCAN_mahalanobis = HDBSCAN(min_cluster_size=15 , cluster_selection_method='eom', prediction_data=True, metric='mahalanobis')
count_vectorizer_model = CountVectorizer(
    ngram_range=(1, 2),
    max_df=0.95,                # исключение слишком частых терминов
    min_df=0.01,                # редкие
    max_features=10000          # размер словаря
)
ctfidf_transformer_model = ClassTfidfTransformer()
representation_model_kbert = KeyBERTInspired()
def create_topic_model_dist(docs, hdb):
    return BERTopic(language="russian", embedding_model=embedding_model_sbert, umap_model=umap_model_cosine, hdbscan_model=hdb, vectorizer_model=count_vectorizer_model, ctfidf_model=ctfidf_transformer_model, representation_model=representation_model_kbert).fit(docs)


### Предложения

#### Euclidean

In [12]:
euclidean_model = create_topic_model_dist(sentences['text'].to_list(), HDBSCAN_euclidean)
euclidean_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6489,-1_авдотья романовна_петр петрович_порфирий пе...,"[авдотья романовна, петр петрович, порфирий пе...",[понимать катерина ивановна понима...
1,0,1047,0_мысль_обстоятельство_случай_чувствовать,"[мысль, обстоятельство, случай, чувствовать, н...",[понимать ясно ощущать весь сила ...
2,1,582,1_отворять дверь_отворять настежь_запирать две...,"[отворять дверь, отворять настежь, запирать дв...",[комната душно окно отворять ...
3,2,490,2_спрашивать раскольников_отвечать раскольнико...,"[спрашивать раскольников, отвечать раскольнико...","[спрашивать раскольников, хотеть отвечать ..."
4,3,335,3____,"[, , , , , , , , , ]","[, , ]"
...,...,...,...,...,...
102,101,16,101_надворный_надворный советник_отказывать_ив...,"[надворный, надворный советник, отказывать, ив...",[мнительный самолюбивый знающий цена ...
103,102,16,102_ах_ах ах_ох_,"[ах, ах ах, ох, , , , , , , ]","[ах, ах, ах]"
104,103,15,103_ничто понимать_понимать ничто_понимать пон...,"[ничто понимать, понимать ничто, понимать пони...","[ничто понимать, н ничто п..."
105,104,15,104_цель_задача_намерение_намерение мочь,"[цель, задача, намерение, намерение мочь, мочь...","[цель, цель разблаготворяться, цель]"


In [13]:
euclidean_model.visualize_barchart()

In [14]:
euclidean_model.visualize_topics()

In [15]:
euclidean_model.visualize_hierarchy()

In [16]:
euclidean_model.visualize_term_rank()

In [17]:
euclidean_model.visualize_heatmap()


#### Chebyshev

In [18]:
cheb_model = create_topic_model_dist(sentences['text'].to_list(), HDBSCAN_chebyshev)

In [19]:
cheb_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,7523,-1_авдотья романовна_романовна_ивановна_петр п...,"[авдотья романовна, романовна, ивановна, петр ...",[склонять голова закрывать рука лицо...
1,0,498,0_отвечать раскольников_раскольников отвечать_...,"[отвечать раскольников, раскольников отвечать,...","[проговаривать раскольников, отвечать ра..."
2,1,335,1____,"[, , , , , , , , , ]","[, , ]"
3,2,234,2_поступок_случай_рассудок_невозможность,"[поступок, случай, рассудок, невозможность, об...",[кеплеров ньютонов открытие вследствие...
4,3,218,3_кричать_кругом кричать_кричать голос_кричать...,"[кричать, кругом кричать, кричать голос, крича...","[кричать толпа многие голос, кричать..."
...,...,...,...,...,...
111,110,16,110_стоять_настасья стоять_это стоять_колено с...,"[стоять, настасья стоять, это стоять, колено с...","[стоять, стоять, стоять]"
112,111,15,111_великолепно_замечательно_прекрасно_молодец,"[великолепно, замечательно, прекрасно, молодец...","[это казаться великолепно, великолепно, ..."
113,112,15,112_иванович вахрушин_почта разбивать_господин...,"[иванович вахрушин, почта разбивать, господин ...",[иной муж али юноша воображать ...
114,113,15,113_надежда мочь_надеяться_мочь надеяться_весь...,"[надежда мочь, надеяться, мочь надеяться, весь...","[говорить надежда смотреть, пора ..."


In [20]:
cheb_model.visualize_topics()

In [21]:
cheb_model.visualize_hierarchy()

In [22]:
cheb_model.visualize_term_rank()

In [23]:
cheb_model.visualize_barchart()

In [24]:
cheb_model.visualize_heatmap()

#### Manhattan

In [25]:
manh_model = create_topic_model_dist(sentences['text'].to_list(), HDBSCAN_manhattan)

In [26]:
manh_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,7266,-1_авдотья романовна_софья семеновна_входить_р...,"[авдотья романовна, софья семеновна, входить, ...",[гм значит решать окончательно д...
1,0,841,0_случай_преступление_обстоятельство_сознание,"[случай, преступление, обстоятельство, сознани...",[приходить мало помалу многообразный ...
2,1,487,1_вскричать раскольников_проговаривать расколь...,"[вскричать раскольников, проговаривать расколь...","[случаться отвечать раскольников, хотеть ..."
3,2,335,2____,"[, , , , , , , , , ]","[, , ]"
4,3,175,3_платье_платьице_белье_одеваться,"[платье, платьице, белье, одеваться, костюм, н...",[одевать темненький легкий м...
...,...,...,...,...,...
107,106,16,106_ах_ах ах_ох_,"[ах, ах ах, ох, , , , , , , ]","[ах, ах, ах]"
108,107,16,107_проходить минута_минута проходить_время пр...,"[проходить минута, минута проходить, время про...","[проходить минута пять, проходить минута..."
109,108,16,108_рог_пес_собака_крест,"[рог, пес, собака, крест, крюк, серьга, лазарь...","[рог, рог, рог]"
110,109,16,109_изволить знать_знать мочь_это узнавать_узн...,"[изволить знать, знать мочь, это узнавать, узн...","[изволить знать, изволить знать, изволить ..."


In [27]:
manh_model.visualize_topics()

In [28]:
manh_model.visualize_hierarchy()

In [29]:
manh_model.visualize_term_rank()

In [30]:
manh_model.visualize_barchart()

In [31]:
manh_model.visualize_heatmap()

#### Mahalanobis

In [32]:
mah_model = create_topic_model_dist(sentences['text'].to_list(), HDBSCAN_chebyshev)

In [33]:
mah_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,7147,-1_авдотья романовна_петр петрович_ивановна_ро...,"[авдотья романовна, петр петрович, ивановна, р...",[ужасно рад уходить пойти потупи...
1,0,800,0_обстоятельство_случай_конец_чувствовать,"[обстоятельство, случай, конец, чувствовать, м...",[приходить мало помалу многообразный ...
2,1,556,1_отвечать раскольников_вскричать раскольников...,"[отвечать раскольников, вскричать раскольников...","[хотеть отвечать раскольников, отвечать ..."
3,2,336,2____,"[, , , , , , , , , ]","[, , ]"
4,3,268,3_деньги_платить_кошелек_весь деньги,"[деньги, платить, кошелек, весь деньги, капита...","[деньги, деньги, деньги]"
...,...,...,...,...,...
100,99,16,99_петр петрович_петрович тотчас_ах петр_петро...,"[петр петрович, петрович тотчас, ах петр, петр...","[петр петрович струшивать, петр петров..."
101,100,16,100_почему_почему почему_отчего_почему именно,"[почему, почему почему, отчего, почему именно,...","[почему, почему, почему почему]"
102,101,16,101_проходить минута_минута проходить_время пр...,"[проходить минута, минута проходить, время про...","[проходить минута пять, проходить мину..."
103,102,15,102_отвечать_отвечать дуня_отвечать дунечка_ду...,"[отвечать, отвечать дуня, отвечать дунечка, ду...","[отвечать, отвечать, отвечать]"


In [34]:
mah_model.visualize_topics()

In [35]:
mah_model.visualize_hierarchy()

In [36]:
mah_model.visualize_term_rank()

In [37]:
mah_model.visualize_barchart()

In [38]:
mah_model.visualize_heatmap()

### Абзацы

#### Euclidean

In [39]:
articles_euclidean_model = create_topic_model_dist(articles['text'].to_list(), HDBSCAN_euclidean)
articles_euclidean_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2085,-1_проговаривать_тотчас_случай_свидригайлов,"[проговаривать, тотчас, случай, свидригайлов, ...",[становиться больной сказать разумих...
1,0,264,0_отвечать раскольников_пробормотать раскольни...,"[отвечать раскольников, пробормотать раскольни...","[хотеть отвечать раскольников, раскольнико..."
2,1,202,1_авдотья романовна_софья семеновна_обращаться...,"[авдотья романовна, софья семеновна, обращатьс...",[петр петрович разменивать причина...
3,2,127,2_чувствовать_умирать_отчаяние_припоминать,"[чувствовать, умирать, отчаяние, припоминать, ...",[однако беспамятство вес...
4,3,105,3_прошептать соня_вскричать соня_пробормотать ...,"[прошептать соня, вскричать соня, пробормотать...","[понимать прошептать соня, прошептать со..."
5,4,80,4_крикнуть катерина_катерина ивановна_авдотья ...,"[крикнуть катерина, катерина ивановна, авдотья...",[катерина ивановна отдышаться время ...
6,5,60,5_спрашивать_спрашивать спрашивать_спрашивать ...,"[спрашивать, спрашивать спрашивать, спрашивать...",[вовсе хотеться спрашивать спр...
7,6,57,6_тотчас_понимать_прочее_случай,"[тотчас, понимать, прочее, случай, давать, ама...",[это самый видеть весь это ...
8,7,51,7_отворять_входить_проходить_проходить мимо,"[отворять, входить, проходить, проходить мимо,...",[останавливаться раздумье ворота идт...
9,8,45,8_вскричать пульхерия_пульхерия александровна_...,"[вскричать пульхерия, пульхерия александровна,...",[боже вскричать пульхерия александровн...


In [40]:
articles_euclidean_model.visualize_barchart()

In [41]:
articles_euclidean_model.visualize_topics()

In [42]:
articles_euclidean_model.visualize_hierarchy()

In [43]:
articles_euclidean_model.visualize_term_rank()

In [44]:
articles_euclidean_model.visualize_heatmap()


#### Chebyshev

In [45]:
articles_cheb_model = create_topic_model_dist(articles['text'].to_list(), HDBSCAN_chebyshev)

In [46]:
articles_cheb_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2160,-1_проговаривать_вставать_тотчас_мысль,"[проговаривать, вставать, тотчас, мысль, разум...",[видемша начинать мещанин дворник ...
1,0,288,0_отвечать раскольников_раскольников отвечать_...,"[отвечать раскольников, раскольников отвечать,...","[хотеть отвечать раскольников, раскольнико..."
2,1,182,1_авдотья романовна_софья семеновна_господин л...,"[авдотья романовна, софья семеновна, господин ...",[это начинать сегодня ве...
3,2,107,2_авдотья романовна_тотчас_понимать_прочее,"[авдотья романовна, тотчас, понимать, прочее, ...",[это самый видеть весь это ...
4,3,100,3_прошептать соня_вскричать соня_вскрикивать с...,"[прошептать соня, вскричать соня, вскрикивать ...","[понимать прошептать соня, прошептать со..."
5,4,59,4_спрашивать_спрашивать спрашивать_спрашивать ...,"[спрашивать, спрашивать спрашивать, спрашивать...","[николай спрашивать, вовсе хотеться ..."
6,5,57,5_авдотья романовна_продолжать раскольников_ка...,"[авдотья романовна, продолжать раскольников, к...",[долой квартира марш слово н...
7,6,56,6_плакать обнимать_забываться_соня чувствовать...,"[плакать обнимать, забываться, соня чувствоват...",[странный неожиданный ощущение едкий...
8,7,46,7_отворять_входить_проходить_проходить мимо,"[отворять, входить, проходить, проходить мимо,...",[полный отчаяние пойти прямо навст...
9,8,46,8_ничто понимать_дескать бред_сочинять_ничто,"[ничто понимать, дескать бред, сочинять, ничто...",[н ничто понимать сказать он...


In [47]:
articles_cheb_model.visualize_topics()

In [48]:
articles_cheb_model.visualize_hierarchy()

In [49]:
articles_cheb_model.visualize_term_rank()

In [50]:
articles_cheb_model.visualize_barchart()

In [51]:
articles_cheb_model.visualize_heatmap()

#### Manhattan

In [52]:
articles_manh_model = create_topic_model_dist(articles['text'].to_list(), HDBSCAN_manhattan)

In [53]:
articles_manh_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2058,-1_проговаривать_мысль_прощать_разумихин,"[проговаривать, мысль, прощать, разумихин, слу...",[катерина ивановна стоять место ...
1,0,262,0_отвечать раскольников_раскольников отвечать_...,"[отвечать раскольников, раскольников отвечать,...","[отвечать раскольников, отвечать раскольни..."
2,1,229,1_авдотья романовна_софья семеновна_взять день...,"[авдотья романовна, софья семеновна, взять ден...",[авдотья романовна однажды повидаться ...
3,2,123,2_вскричать соня_прошептать соня_вскрикивать с...,"[вскричать соня, прошептать соня, вскрикивать ...","[понимать прошептать соня, н прошептат..."
4,3,115,3_авдотья романовна_тотчас_понимать_прочее,"[авдотья романовна, тотчас, понимать, прочее, ...",[это самый видеть весь это ...
5,4,80,4_авдотья романовна_катерина ивановна_вскрикив...,"[авдотья романовна, катерина ивановна, вскрики...",[мармеладов последний агония отв...
6,5,75,5_замирать_забываться_любить_чувство,"[замирать, забываться, любить, чувство, чувств...",[выходить соня смотреть помешанн...
7,6,58,6_спрашивать_спрашивать спрашивать_спрашивать ...,"[спрашивать, спрашивать спрашивать, спрашивать...","[николай спрашивать, вовсе хотеться ..."
8,7,55,7_отворять_входить_проходить_выходить канава,"[отворять, входить, проходить, выходить канава...",[полный отчаяние пойти прямо навст...
9,8,43,8_вскричать пульхерия_александровна пульхерия_...,"[вскричать пульхерия, александровна пульхерия,...",[боже вскричать пульхерия александровн...


In [54]:
articles_manh_model.visualize_topics()

In [55]:
articles_manh_model.visualize_hierarchy()

In [56]:
articles_manh_model.visualize_term_rank()

In [57]:
articles_manh_model.visualize_barchart()

In [58]:
articles_manh_model.visualize_heatmap()

#### Mahalanobis

In [59]:
articles_mah_model = create_topic_model_dist(articles['text'].to_list(), HDBSCAN_chebyshev)

In [60]:
articles_mah_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2120,-1_проговаривать_мысль_тотчас_случай,"[проговаривать, мысль, тотчас, случай, понимат...",[входить соня начинаться сумерки в...
1,0,252,0_отвечать раскольников_раскольников отвечать_...,"[отвечать раскольников, раскольников отвечать,...","[отвечать раскольников, отвечать раскольни..."
2,1,206,1_авдотья романовна_софья семеновна_взять день...,"[авдотья романовна, софья семеновна, взять ден...",[софья семеновна мочь америка уезжат...
3,2,110,2_авдотья романовна_тотчас_понимать_прочее,"[авдотья романовна, тотчас, понимать, прочее, ...",[это самый видеть весь это ...
4,3,110,3_прошептать соня_вскричать соня_вскрикивать с...,"[прошептать соня, вскричать соня, вскрикивать ...","[понимать прошептать соня, н прошептат..."
5,4,84,4_замирать_беспамятство_просыпаться_чувствовать,"[замирать, беспамятство, просыпаться, чувствов...",[выходить соня смотреть помешанн...
6,5,77,5_катерина ивановна_авдотья романовна_катерина...,"[катерина ивановна, авдотья романовна, катерин...",[мармеладов последний агония отв...
7,6,56,6_спрашивать_спрашивать спрашивать_разумихин с...,"[спрашивать, спрашивать спрашивать, разумихин ...","[николай спрашивать, вовсе хотеться ..."
8,7,48,7_отворять_входить_проходить_проходить мимо,"[отворять, входить, проходить, проходить мимо,...",[полный отчаяние пойти прямо навст...
9,8,42,8_вскричать пульхерия_вскрикивать пульхерия_пу...,"[вскричать пульхерия, вскрикивать пульхерия, п...",[любить проговаривать растрогать пульхер...


## Сравнение эффективности и метрики

In [61]:
from sklearn.metrics import calinski_harabasz_score, silhouette_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
import numpy as np

In [69]:
vectorizer = CountVectorizer(max_features=5000)
data = sentences['text']
X = vectorizer.fit_transform(data)

#сделаем векторное представление данных
vectorizer_model = CountVectorizer()
vectorizer_model.fit(data)
document_vectors = vectorizer_model.transform(data).toarray()

# PCA для уменьшения размерности
pca = PCA(n_components=5)
reduced_document_vectors = pca.fit_transform(document_vectors)

topics1, probs = euclidean_model.fit_transform(data)
topics2, probs = cheb_model.fit_transform(data)
topics3, probs = manh_model.fit_transform(data)
# topics4, probs = mah_model.transform(data)


In [70]:
print(f"""1 model {calinski_harabasz_score(reduced_document_vectors, topics1)}
2 model {calinski_harabasz_score(reduced_document_vectors, topics2)}
3 model {calinski_harabasz_score(reduced_document_vectors, topics3)}""")

1 model 17.496272037444392
2 model 16.393289097626027
3 model 18.401579166054532


In [73]:
print(f"""1 model {silhouette_score(reduced_document_vectors, topics1)}
2 model {silhouette_score(reduced_document_vectors, topics2)}
3 model {silhouette_score(reduced_document_vectors, topics3)}""")

1 model -0.613438420443327
2 model -0.6208699345651668
3 model -0.6135714055230438
